# ROS Bag Data Visualization - Single Run

This notebook provides visualization for a single ROS bag run.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

csv_file_path = '../downloaded_files/run-2025-09-14-171046/rooms-generated-p1/0/rosbag2.csv'
print(f'Looking for data file: {csv_file_path}')

df = pd.read_csv(csv_file_path)
print(f'Loaded {len(df)} rows from {csv_file_path}')

print(f'Dataset shape: {df.shape}')
print(f'Column names: {list(df.columns)}')

In [ ]:
# Filter the data for Gazebo real-time factor
rtf_data = df[df['topic'] == '/gazebo/real_time_factor'].copy()

if not rtf_data.empty:
    # Extract real-time factor values and timestamps
    rtf_values = rtf_data['value'].astype(float)
    rtf_timestamps = rtf_data['timestamp'].astype(float)
    
    print(f"Found {len(rtf_data)} real-time factor data points")
    print(f"RTF range: {rtf_values.min():.4f} to {rtf_values.max():.4f}")
    print(f"Mean RTF: {rtf_values.mean():.4f}")
    print(f"Std RTF: {rtf_values.std():.4f}")
    
    # Create subplots for different views
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Time series plot
    ax1.plot(rtf_timestamps, rtf_values, 'b-', linewidth=1, alpha=0.7)
    ax1.axhline(y=1.0, color='r', linestyle='--', alpha=0.8, label='Ideal RTF = 1.0')
    ax1.fill_between(rtf_timestamps, rtf_values, alpha=0.3)
    ax1.set_xlabel('Time (seconds)')
    ax1.set_ylabel('Real-Time Factor')
    ax1.set_title('Real-Time Factor Over Time')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Add statistics text
    stats_text = f'Mean: {rtf_values.mean():.4f}\\nStd: {rtf_values.std():.4f}\\nMin: {rtf_values.min():.4f}\\nMax: {rtf_values.max():.4f}'
    ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # 2. Histogram
    ax2.hist(rtf_values, bins=50, alpha=0.7, edgecolor='black', linewidth=0.5)
    ax2.axvline(x=1.0, color='r', linestyle='--', alpha=0.8, label='Ideal RTF = 1.0')
    ax2.axvline(x=rtf_values.mean(), color='g', linestyle='-', alpha=0.8, label=f'Mean: {rtf_values.mean():.4f}')
    ax2.set_xlabel('Real-Time Factor')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Distribution of Real-Time Factor')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Performance zones
    excellent = (rtf_values >= 0.95) & (rtf_values <= 1.05)
    good = ((rtf_values >= 0.8) & (rtf_values < 0.95)) | ((rtf_values > 1.05) & (rtf_values <= 1.2))
    poor = (rtf_values < 0.8) | (rtf_values > 1.2)
    
    ax3.scatter(rtf_timestamps[excellent], rtf_values[excellent], 
               c='green', s=20, alpha=0.7, label=f'Excellent (0.95-1.05): {excellent.sum()}')
    ax3.scatter(rtf_timestamps[good], rtf_values[good], 
               c='orange', s=20, alpha=0.7, label=f'Good (0.8-0.95, 1.05-1.2): {good.sum()}')
    ax3.scatter(rtf_timestamps[poor], rtf_values[poor], 
               c='red', s=20, alpha=0.7, label=f'Poor (<0.8, >1.2): {poor.sum()}')
    
    ax3.axhline(y=1.0, color='black', linestyle='-', alpha=0.5)
    ax3.axhline(y=0.95, color='green', linestyle='--', alpha=0.5)
    ax3.axhline(y=1.05, color='green', linestyle='--', alpha=0.5)
    ax3.axhline(y=0.8, color='orange', linestyle='--', alpha=0.5)
    ax3.axhline(y=1.2, color='orange', linestyle='--', alpha=0.5)
    
    ax3.set_xlabel('Time (seconds)')
    ax3.set_ylabel('Real-Time Factor')
    ax3.set_title('Performance Zones')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Moving average
    window_size = max(10, len(rtf_values) // 50)  # Adaptive window size
    moving_avg = rtf_values.rolling(window=window_size, center=True).mean()
    
    ax4.plot(rtf_timestamps, rtf_values, 'b-', alpha=0.3, linewidth=0.5, label='Raw data')
    ax4.plot(rtf_timestamps, moving_avg, 'r-', linewidth=2, label=f'Moving avg ({window_size} points)')
    ax4.axhline(y=1.0, color='black', linestyle='--', alpha=0.8, label='Ideal RTF = 1.0')
    ax4.fill_between(rtf_timestamps, moving_avg - rtf_values.std(), 
                     moving_avg + rtf_values.std(), alpha=0.2, label='±1σ band')
    
    ax4.set_xlabel('Time (seconds)')
    ax4.set_ylabel('Real-Time Factor')
    ax4.set_title('Real-Time Factor Trend')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Performance summary
    print("\\n" + "="*50)
    print("PERFORMANCE SUMMARY")
    print("="*50)
    print(f"Excellent performance (0.95-1.05): {excellent.sum()/len(rtf_values)*100:.1f}% of time")
    print(f"Good performance (0.8-0.95, 1.05-1.2): {good.sum()/len(rtf_values)*100:.1f}% of time")
    print(f"Poor performance (<0.8, >1.2): {poor.sum()/len(rtf_values)*100:.1f}% of time")
    
    # Check for significant drops
    significant_drops = rtf_values < 0.1
    if significant_drops.any():
        print(f"\\nWarning: {significant_drops.sum()} significant performance drops detected (RTF < 0.1)")
        drop_times = rtf_timestamps[significant_drops]
        print(f"Drop times: {drop_times.values}")
        
else:
    print("No real-time factor data found in the CSV file")
    print("Available topics:", df['topic'].unique())

# System Resource Monitoring

Analyzing system performance through CPU and memory usage metrics during the simulation run.

In [ ]:
# Filter the data for CPU and memory usage
cpu_data = df[df['topic'] == '/system/cpu_usage'].copy()
memory_data = df[df['topic'] == '/system/memory_usage'].copy()

if not cpu_data.empty or not memory_data.empty:
    # Create figure with subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Process CPU data
    if not cpu_data.empty:
        cpu_values = cpu_data['value'].astype(float)
        cpu_timestamps = cpu_data['timestamp'].astype(float)
        
        print(f"Found {len(cpu_data)} CPU usage data points")
        print(f"CPU usage range: {cpu_values.min():.1f}% to {cpu_values.max():.1f}%")
        print(f"Mean CPU usage: {cpu_values.mean():.1f}%")
        print(f"Std CPU usage: {cpu_values.std():.1f}%")
        
        # CPU time series
        axes[0,0].plot(cpu_timestamps, cpu_values, 'b-', linewidth=1.5, alpha=0.8)
        axes[0,0].fill_between(cpu_timestamps, cpu_values, alpha=0.3, color='blue')
        axes[0,0].axhline(y=cpu_values.mean(), color='r', linestyle='--', alpha=0.8, 
                         label=f'Mean: {cpu_values.mean():.1f}%')
        axes[0,0].set_xlabel('Time (seconds)')
        axes[0,0].set_ylabel('CPU Usage (%)')
        axes[0,0].set_title('CPU Usage Over Time')
        axes[0,0].grid(True, alpha=0.3)
        axes[0,0].legend()
        axes[0,0].set_ylim(0, 100)
        
        # Add statistics text for CPU
        cpu_stats = f'Mean: {cpu_values.mean():.1f}%\\nStd: {cpu_values.std():.1f}%\\nMin: {cpu_values.min():.1f}%\\nMax: {cpu_values.max():.1f}%'
        axes[0,0].text(0.02, 0.98, cpu_stats, transform=axes[0,0].transAxes, 
                       verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
        
        # CPU histogram
        axes[0,1].hist(cpu_values, bins=30, alpha=0.7, color='blue', edgecolor='black', linewidth=0.5)
        axes[0,1].axvline(x=cpu_values.mean(), color='r', linestyle='-', alpha=0.8, 
                         label=f'Mean: {cpu_values.mean():.1f}%')
        axes[0,1].axvline(x=cpu_values.median(), color='g', linestyle='--', alpha=0.8, 
                         label=f'Median: {cpu_values.median():.1f}%')
        axes[0,1].set_xlabel('CPU Usage (%)')
        axes[0,1].set_ylabel('Frequency')
        axes[0,1].set_title('CPU Usage Distribution')
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)
        
        # CPU usage zones
        low_cpu = cpu_values < 25
        moderate_cpu = (cpu_values >= 25) & (cpu_values < 50)
        high_cpu = (cpu_values >= 50) & (cpu_values < 75)
        critical_cpu = cpu_values >= 75
        
        axes[0,2].scatter(cpu_timestamps[low_cpu], cpu_values[low_cpu], 
                         c='green', s=20, alpha=0.7, label=f'Low (<25%): {low_cpu.sum()}')
        axes[0,2].scatter(cpu_timestamps[moderate_cpu], cpu_values[moderate_cpu], 
                         c='yellow', s=20, alpha=0.7, label=f'Moderate (25-50%): {moderate_cpu.sum()}')
        axes[0,2].scatter(cpu_timestamps[high_cpu], cpu_values[high_cpu], 
                         c='orange', s=20, alpha=0.7, label=f'High (50-75%): {high_cpu.sum()}')
        axes[0,2].scatter(cpu_timestamps[critical_cpu], cpu_values[critical_cpu], 
                         c='red', s=20, alpha=0.7, label=f'Critical (>75%): {critical_cpu.sum()}')
        
        axes[0,2].axhline(y=25, color='green', linestyle='--', alpha=0.5)
        axes[0,2].axhline(y=50, color='yellow', linestyle='--', alpha=0.5)
        axes[0,2].axhline(y=75, color='red', linestyle='--', alpha=0.5)
        
        axes[0,2].set_xlabel('Time (seconds)')
        axes[0,2].set_ylabel('CPU Usage (%)')
        axes[0,2].set_title('CPU Usage Zones')
        axes[0,2].legend()
        axes[0,2].grid(True, alpha=0.3)
        axes[0,2].set_ylim(0, 100)
    else:
        # Hide CPU plots if no data
        for ax in axes[0,:]:
            ax.set_visible(False)
        print("No CPU usage data found")
    
    # Process Memory data
    if not memory_data.empty:
        memory_values = memory_data['value'].astype(float)
        memory_timestamps = memory_data['timestamp'].astype(float)
        
        print(f"\\nFound {len(memory_data)} memory usage data points")
        print(f"Memory usage range: {memory_values.min():.1f}% to {memory_values.max():.1f}%")
        print(f"Mean memory usage: {memory_values.mean():.1f}%")
        print(f"Std memory usage: {memory_values.std():.1f}%")
        
        # Memory time series
        axes[1,0].plot(memory_timestamps, memory_values, 'g-', linewidth=1.5, alpha=0.8)
        axes[1,0].fill_between(memory_timestamps, memory_values, alpha=0.3, color='green')
        axes[1,0].axhline(y=memory_values.mean(), color='r', linestyle='--', alpha=0.8, 
                         label=f'Mean: {memory_values.mean():.1f}%')
        axes[1,0].set_xlabel('Time (seconds)')
        axes[1,0].set_ylabel('Memory Usage (%)')
        axes[1,0].set_title('Memory Usage Over Time')
        axes[1,0].grid(True, alpha=0.3)
        axes[1,0].legend()
        axes[1,0].set_ylim(0, 100)
        
        # Add statistics text for memory
        mem_stats = f'Mean: {memory_values.mean():.1f}%\\nStd: {memory_values.std():.1f}%\\nMin: {memory_values.min():.1f}%\\nMax: {memory_values.max():.1f}%'
        axes[1,0].text(0.02, 0.98, mem_stats, transform=axes[1,0].transAxes, 
                       verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
        
        # Memory histogram
        axes[1,1].hist(memory_values, bins=30, alpha=0.7, color='green', edgecolor='black', linewidth=0.5)
        axes[1,1].axvline(x=memory_values.mean(), color='r', linestyle='-', alpha=0.8, 
                         label=f'Mean: {memory_values.mean():.1f}%')
        axes[1,1].axvline(x=memory_values.median(), color='b', linestyle='--', alpha=0.8, 
                         label=f'Median: {memory_values.median():.1f}%')
        axes[1,1].set_xlabel('Memory Usage (%)')
        axes[1,1].set_ylabel('Frequency')
        axes[1,1].set_title('Memory Usage Distribution')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
        
        # Memory usage zones
        low_mem = memory_values < 30
        moderate_mem = (memory_values >= 30) & (memory_values < 60)
        high_mem = (memory_values >= 60) & (memory_values < 85)
        critical_mem = memory_values >= 85
        
        axes[1,2].scatter(memory_timestamps[low_mem], memory_values[low_mem], 
                         c='green', s=20, alpha=0.7, label=f'Low (<30%): {low_mem.sum()}')
        axes[1,2].scatter(memory_timestamps[moderate_mem], memory_values[moderate_mem], 
                         c='yellow', s=20, alpha=0.7, label=f'Moderate (30-60%): {moderate_mem.sum()}')
        axes[1,2].scatter(memory_timestamps[high_mem], memory_values[high_mem], 
                         c='orange', s=20, alpha=0.7, label=f'High (60-85%): {high_mem.sum()}')
        axes[1,2].scatter(memory_timestamps[critical_mem], memory_values[critical_mem], 
                         c='red', s=20, alpha=0.7, label=f'Critical (>85%): {critical_mem.sum()}')
        
        axes[1,2].axhline(y=30, color='green', linestyle='--', alpha=0.5)
        axes[1,2].axhline(y=60, color='yellow', linestyle='--', alpha=0.5)
        axes[1,2].axhline(y=85, color='red', linestyle='--', alpha=0.5)
        
        axes[1,2].set_xlabel('Time (seconds)')
        axes[1,2].set_ylabel('Memory Usage (%)')
        axes[1,2].set_title('Memory Usage Zones')
        axes[1,2].legend()
        axes[1,2].grid(True, alpha=0.3)
        axes[1,2].set_ylim(0, 100)
    else:
        # Hide memory plots if no data
        for ax in axes[1,:]:
            ax.set_visible(False)
        print("No memory usage data found")
    
    plt.tight_layout()
    plt.show()
    
    # Performance summary
    print("\\n" + "="*60)
    print("SYSTEM RESOURCE SUMMARY")
    print("="*60)
    
    if not cpu_data.empty:
        cpu_low_time = (cpu_values < 25).sum() / len(cpu_values) * 100
        cpu_mod_time = ((cpu_values >= 25) & (cpu_values < 50)).sum() / len(cpu_values) * 100
        cpu_high_time = ((cpu_values >= 50) & (cpu_values < 75)).sum() / len(cpu_values) * 100
        cpu_crit_time = (cpu_values >= 75).sum() / len(cpu_values) * 100
        
        print(f"CPU Usage Analysis:")
        print(f"  Low usage (<25%): {cpu_low_time:.1f}% of time")
        print(f"  Moderate usage (25-50%): {cpu_mod_time:.1f}% of time")
        print(f"  High usage (50-75%): {cpu_high_time:.1f}% of time")
        print(f"  Critical usage (>75%): {cpu_crit_time:.1f}% of time")
    
    if not memory_data.empty:
        mem_low_time = (memory_values < 30).sum() / len(memory_values) * 100
        mem_mod_time = ((memory_values >= 30) & (memory_values < 60)).sum() / len(memory_values) * 100
        mem_high_time = ((memory_values >= 60) & (memory_values < 85)).sum() / len(memory_values) * 100
        mem_crit_time = (memory_values >= 85).sum() / len(memory_values) * 100
        
        print(f"\\nMemory Usage Analysis:")
        print(f"  Low usage (<30%): {mem_low_time:.1f}% of time")
        print(f"  Moderate usage (30-60%): {mem_mod_time:.1f}% of time")
        print(f"  High usage (60-85%): {mem_high_time:.1f}% of time")
        print(f"  Critical usage (>85%): {mem_crit_time:.1f}% of time")
        
        # Check for memory growth trend
        if len(memory_values) > 10:
            # Simple linear trend analysis
            x = np.arange(len(memory_values))
            slope = np.polyfit(x, memory_values, 1)[0]
            if abs(slope) > 0.1:  # Threshold for significant trend
                trend_direction = "increasing" if slope > 0 else "decreasing"
                print(f"\\nMemory trend: {trend_direction} at {slope:.2f}%/measurement")
                if slope > 0.5:
                    print("  ⚠️  Warning: Potential memory leak detected!")
else:
    print("No system resource data found in the CSV file")
    print("Available topics:", df['topic'].unique())